In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/0-tratamiento-correcto-columnas'

/content/drive/My Drive/Colab Notebooks/0-tratamiento-correcto-columnas


In [ ]:
import os
import random as rd
import numpy as np
import pandas as pd

In [ ]:
ruta_iniciales = os.path.join(os.getcwd(), 'archivos-iniciales')
ruta_resultados = os.path.join(os.getcwd(), 'archivos-resultados')

In [ ]:
archivos_iniciales = os.listdir(ruta_iniciales)
archivos_iniciales

['data-ct-arroz.csv', 'data-ct-maiz.csv', 'data-cp-cacao-ramilla.csv']

**MAIZ**

### CRITERIO HECTAREA MAYOR

In [ ]:
ruta_maiz = os.path.join(ruta_iniciales, 'data-ct-maiz.csv')
maiz = pd.read_csv(ruta_maiz)
maiz

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,eu_trc,eu_tro,eu_tn,...,ct_cantidad_fun_pq,ct_umed_fun_pq,ct_color_fun_pq,ct_cantidad_pq,ct_umed_pq,ct_color_pq,ct_trabocacont,ct_trabfami,ct_piensasem_int,ct_supersem_int,ct_supersemha_int,ct_messiem_int,ct_condcult_int,ct_codcultiv1_int,ct_codcultiv2_int,ct_nclavr,ct_trs,ct_trc,ct_tro,ct_tn,ct_prod,ct_superdida,superdida,ct_sequia,ct_helada,ct_plagas,ct_enfermedades,ct_inundacion,ct_otra,ct_secomun,ct_semejorada,ct_secertificada,ct_sehibrinac,ct_sehibrinter,Ventas,ct_sriego,su_fertilizada,su_plaguicidas,ct_nuevacondicion,fact_exp_fin_cultivo
0,12085002069670021,12,8,50,2,6967,21,1,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,28.0000,22,22.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,20.0,20.0,NaN,NaN,2.0,NaN,4,...,6.0,5.0,3.0,NaN,NaN,NaN,10.0,NaN,1,28.00,28.0000,6.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,165.454545,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.00,NaN,NaN,165.454545,28.0,28.0000,28.0000,1,0.048231
1,12085002069670021,12,8,50,2,6967,21,1,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,28.0000,22,22.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,20.0,20.0,NaN,NaN,2.0,NaN,4,...,6.0,5.0,3.0,NaN,NaN,NaN,10.0,NaN,1,28.00,28.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,178.181818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.00,NaN,NaN,177.272727,NaN,28.0000,28.0000,1,0.048231
2,24015303001930003,24,1,53,3,193,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5200.0000,1,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,120.00,120.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,490.909091,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.00,NaN,NaN,490.909091,120.0,120.0000,120.0000,1,2.047315
3,12085002082940005,12,8,50,2,8294,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,16.0000,15,15.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.00,1.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,2.727273,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,2.727273,NaN,1.0000,1.0000,1,1.602667
4,12075404000020004,12,7,54,4,2,4,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,50.0000,4,4.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1,50.00,50.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,318.181818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,318.181818,NaN,50.0000,50.0000,1,0.152810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2279,12085002083060004,12,8,50,2,8306,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7056,1,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.95,0.6703,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,1.818182,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6703,NaN,NaN,NaN,NaN,1.681818,NaN,0.6703,0.6703,1,297.684506
2280,9145003006170019,9,14,50,3,617,19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.00,1.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,0.681818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,NaN,NaN,NaN,0.681818,NaN,1.0000,1.0000,1,386.421311
2281,9145003006170002,9,14,50,3,617,2,1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0000,3,3.0,NaN,1

In [ ]:
maiz['Identificador'].duplicated().sum()

479

In [ ]:
maiz_res = maiz.drop_duplicates(subset=['Identificador'], keep='first', ignore_index=True)
maiz_res

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,eu_trc,eu_tro,eu_tn,...,ct_cantidad_fun_pq,ct_umed_fun_pq,ct_color_fun_pq,ct_cantidad_pq,ct_umed_pq,ct_color_pq,ct_trabocacont,ct_trabfami,ct_piensasem_int,ct_supersem_int,ct_supersemha_int,ct_messiem_int,ct_condcult_int,ct_codcultiv1_int,ct_codcultiv2_int,ct_nclavr,ct_trs,ct_trc,ct_tro,ct_tn,ct_prod,ct_superdida,superdida,ct_sequia,ct_helada,ct_plagas,ct_enfermedades,ct_inundacion,ct_otra,ct_secomun,ct_semejorada,ct_secertificada,ct_sehibrinac,ct_sehibrinter,Ventas,ct_sriego,su_fertilizada,su_plaguicidas,ct_nuevacondicion,fact_exp_fin_cultivo
0,12085002069670021,12,8,50,2,6967,21,1,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,28.0000,22,22.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,20.0,20.0,NaN,NaN,2.0,NaN,4,...,6.0,5.0,3.0,NaN,NaN,NaN,10.0,NaN,1,28.00,28.0000,6.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,165.454545,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.00,NaN,NaN,165.454545,28.0,28.0000,28.0000,1,0.048231
1,24015303001930003,24,1,53,3,193,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5200.0000,1,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,120.00,120.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,490.909091,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.00,NaN,NaN,490.909091,120.0,120.0000,120.0000,1,2.047315
2,12085002082940005,12,8,50,2,8294,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,16.0000,15,15.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.00,1.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,2.727273,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,2.727273,NaN,1.0000,1.0000,1,1.602667
3,12075404000020004,12,7,54,4,2,4,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,50.0000,4,4.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1,50.00,50.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,318.181818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,318.181818,NaN,50.0000,50.0000,1,0.152810
4,12085002087000001,12,8,50,2,8700,1,1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0000,1,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1,4.00,4.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,12.727273,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0000,NaN,NaN,NaN,12.727273,NaN,4.0000,4.0000,1,0.219531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,12085002083060004,12,8,50,2,8306,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7056,1,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.95,0.6703,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,1.818182,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6703,NaN,NaN,NaN,NaN,1.681818,NaN,0.6703,0.6703,1,297.684506
1801,9145003006170019,9,14,50,3,617,19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,2,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.00,1.0000,1.0,1.0,548.0,NaN,548.0,NaN,2.0,NaN,4.0,0.681818,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,NaN,NaN,NaN,0.681818,NaN,1.0000,1.0000,1,386.421311
1802,9145003006170002,9,14,50,3,617,2,1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0000,3,3.0,NaN,1.0,1.0,NaN,2

In [ ]:
muestra = rd.sample(list(maiz['Identificador'][maiz['Identificador'].duplicated()]), 5)
muestra

[12025202104730003,
 12025202121450001,
 13155002180760011,
 12085002068770006,
 12065102098940010]

In [ ]:
def criterio_hectarea_mayor(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1 and df_repetidos[mascara][columna].nunique() > 1:
      array = df_repetidos[mascara]['ct_k509'].values
      array = np.nan_to_num(array, nan=-np.inf)
      pos_max = array.argmax()
      valor = df_repetidos[mascara][columna].values[pos_max]
      df_unicos.loc[id, columna] = valor
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [ ]:
maiz_res = criterio_hectarea_mayor(maiz, maiz_res, 'ual_parr')
maiz_res = criterio_hectarea_mayor(maiz, maiz_res, 'ct_afecta_prod')
maiz_res = criterio_hectarea_mayor(maiz, maiz_res, 'ct_riego')

In [ ]:
# arroz[arroz['Identificador'].isin(muestra)][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']].sort_values('Identificador')
maiz[maiz['Identificador'] == 22045202013360009][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]

,Identificador,ual_parr,ct_k509,ct_afecta_prod,ct_riego,ct_numl,ct_numt
1631,22045202013360009,52,2.0,6.0,0,1,1
1632,22045202013360009,52,12.0,6.0,0,3,3
1633,22045202013360009,52,2.0,6.0,0,2,3


In [ ]:
# arroz_res[arroz_res['Identificador'].isin(muestra)][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]
maiz_res[maiz_res['Identificador'] == 22045202013360009][['Identificador', 'ual_parr', 'ct_k509', 'ct_afecta_prod', 'ct_riego', 'ct_numl', 'ct_numt']]

,Identificador,ual_parr,ct_k509,ct_afecta_prod,ct_riego,ct_numl,ct_numt
1282,22045202013360009,52,2.0,6.0,0,1,1


### CRITERIO SUMA

In [ ]:
def criterio_suma_valores(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1:
      array = df_repetidos[mascara][columna].values
      array = np.nan_to_num(array, nan=0)
      suma  = array.sum()
      df_unicos.loc[id, columna] = suma
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [ ]:
maiz_res = criterio_suma_valores(maiz, maiz_res, 'Ventas')
maiz_res = criterio_suma_valores(maiz, maiz_res, 'ct_cantidad_npk_fq')

In [ ]:
maiz[maiz['Identificador'] == 22045202013360009][['Identificador', 'Ventas', 'ct_cantidad_npk_fq']]

,Identificador,Ventas,ct_cantidad_npk_fq
1631,22045202013360009,0.363636,NaN
1632,22045202013360009,6.818182,NaN
1633,22045202013360009,6.818182,NaN


In [ ]:
maiz_res[maiz_res['Identificador'] == 22045202013360009][['Identificador', 'Ventas', 'ct_cantidad_npk_fq']]

,Identificador,Ventas,ct_cantidad_npk_fq
1282,22045202013360009,14.0,0.0


### CRITERIO BASTA QUE HAYA UN UNO PARA QUE TODOS SEAN UNO

In [ ]:
def criterio_suma_valores(df_repetidos, df_unicos, columna):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    veces = mascara.sum()
    if veces > 1:
      array = df_repetidos[mascara][columna].values
      array = np.nan_to_num(array, nan=0)
      booleano = np.any(array)
      valor = int(booleano)
      df_unicos.loc[id, columna] = valor
  df_unicos.reset_index(inplace=True)
  return df_unicos

In [ ]:
columnas = ['pa_arealm',	'pa_areempa',	'eq_earado',	'eq_semman',	'eq_emotoguadana',	'eq_ebmanual',	'eq_ebestaciona',	'eq_ecosgranfigr']
for columna in columnas:
  maiz_res = criterio_suma_valores(maiz, maiz_res, columna)

In [ ]:
maiz[maiz['Identificador'] == 22045202013360009][['Identificador'] + columnas]

,Identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
1631,22045202013360009,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0
1632,22045202013360009,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0
1633,22045202013360009,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0


In [ ]:
maiz_res[maiz_res['Identificador'] == 22045202013360009][['Identificador'] + columnas]

,Identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
1282,22045202013360009,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


<hr>

In [ ]:
maiz[maiz['Identificador'] == 22045202013360009][['Identificador', 'eu_superficie_ha']]

,Identificador,eu_superficie_ha
1631,22045202013360009,4.81
1632,22045202013360009,4.81
1633,22045202013360009,4.81


In [ ]:
maiz_res[maiz_res['Identificador'] == 22045202013360009][['Identificador', 'eu_superficie_ha']]

,Identificador,eu_superficie_ha
1282,22045202013360009,4.81


### GUARDAR CAMBIOS

In [55]:
maiz_res.to_csv(os.path.join(ruta_resultados, 'maiz.csv'), index=False, encoding='utf-8')
maiz_res.to_stata(os.path.join(ruta_resultados, 'maiz.dta'), write_index=False)